<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Pivot Tables](03.09-Pivot-Tables.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.10-Working-With-Strings.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# ベクトル化された文字列操作

Python の強みの 1 つは、文字列データの処理と操作が比較的簡単なことです。
Pandas はこれに基づいて構築され、実世界のデータを操作する (読み取り: クリーンアップする) ときに必要なタイプの変更の不可欠な要素となる、包括的な一連の *ベクトル化された文字列操作* を提供します。
このセクションでは、いくつかの Pandas 文字列操作について説明し、それらを使用して、インターネットから収集されたレシピの非常に乱雑なデータセットを部分的にクリーンアップする方法を見ていきます。

## Pandas 文字列操作の紹介

前のセクションで、NumPy や Pandas などのツールが算術演算を一般化して、多くの配列要素に対して同じ演算を簡単かつ迅速に実行できるようにする方法を見てきました。例えば：

In [1]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

操作のこの*ベクトル化*により、データの配列を操作する構文が簡素化されます。配列のサイズや形状について心配する必要はなくなり、実行したい操作についてだけ考えることができます。
文字列の配列の場合、NumPy はそのような単純なアクセスを提供しないため、より冗長なループ構文を使用することになります。

In [2]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

一部のデータを扱うにはこれで十分かもしれませんが、欠損値があると壊れます。
例えば：

In [3]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

Pandas には、ベクトル化された文字列操作に対するこのニーズと、文字列を含む Pandas Series および Index オブジェクトの ``str`` 属性を介して欠落データを正しく処理するための両方のニーズに対応する機能が含まれています。
たとえば、次のデータを使用して Pandas シリーズを作成するとします。

In [4]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

欠落している値をスキップしながら、すべてのエントリを大文字にする単一のメソッドを呼び出すことができるようになりました。

In [5]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

この ``str`` 属性でタブ補完を使用すると、Pandas で使用できるすべてのベクトル化された文字列メソッドが一覧表示されます。

## Pandas 文字列メソッドの表

Python での文字列操作を十分に理解している場合、Pandas の文字列構文のほとんどは十分に直感的であるため、使用可能なメソッドの表をリストするだけでおそらく十分です。いくつかの微妙な点に深く飛び込む前に、ここから始めます。
このセクションの例では、次の一連の名前を使用しています。

In [6]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Python の文字列メソッドに似たメソッド
ほぼすべての Python の組み込み文字列メソッドは、Pandas のベクトル化された文字列メソッドによって反映されます。以下は、Python の文字列メソッドを反映した Pandas の ``str`` メソッドのリストです。

|             |                  |                  |                  |
|-------------|------------------|------------------|------------------|
|``len()`` | ``lower()`` | ``translate()`` | islower() |
|``bright()`` | ``upper()'' | ``startswith()'' | ``isupper()'' |
|``rjust()`` | ``find()`` | ``endswith()`` | ``isnumeric()`` |
|``center()`` | ``rfind()`` | ``isalnum()`` | isdecimal() |
|``zfill()`` |インデックス() | ``isalpha()`` |スプリット() |
| ``strip()`` | ``rindex()`` | ``isdigit()`` | ``rsplit()`` |
|``rstrip()`` | ``capitalize()`` | ``isspace()`` |パーティション() |
|``lstrip()`` | ``swapcase()`` | ``istitle()`` | rpartition() |

これらにはさまざまな戻り値があることに注意してください。 ``lower()`` のように、一連の文字列を返すものもあります:

In [7]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

しかし、他のいくつかは数値を返します:

In [8]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

またはブール値:

In [9]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

さらに、各要素のリストまたはその他の複合値を返すものもあります。

In [10]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

議論を続ける中で、この種の一連のリスト オブジェクトの操作をさらに見ていきます。

### 正規表現を使った方法

さらに、正規表現を受け入れて各文字列要素の内容を調べ、Python の組み込み ``re`` モジュールの API 規則のいくつかに従うメソッドがいくつかあります。

| |メソッド |説明 |
|--------|-------------|
| | ``match()`` |各要素で ``re.match()`` を呼び出し、ブール値を返します。 | |
| | ``extract()`` |各要素で ``re.match()`` を呼び出し、一致したグループを文字列として返します。|
| | ``findall()`` |各要素で ``re.findall()`` を呼び出します |
| | ``replace()`` |パターンの発生を他の文字列に置き換えます|
| | ``contains()`` |各要素で ``re.search()`` を呼び出し、ブール値 | を返します。
| | ``count()`` |パターンの出現回数をカウント|
| |スプリット() | str.split() と同等ですが、正規表現を受け入れます |
| | ``rsplit()`` | ``str.rsplit()`` と同等ですが、正規表現を受け入れます |

これらを使用すると、さまざまな興味深い操作を実行できます。
たとえば、各要素の先頭で連続する文字グループを要求することで、それぞれから名を抽出できます。

In [11]:
monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

または、子音で始まり子音で終わるすべての名前を見つけるなど、より複雑なことを行うこともできます。表現文字:

In [12]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

``Series`` または ``Dataframe`` エントリ全体に正規表現を簡潔に適用する機能により、データの分析とクリーニングに多くの可能性が開かれます。

### その他のメソッド
最後に、その他の便利な操作を可能にするいくつかのその他のメソッドがあります。

| |メソッド |説明 |
|--------|-------------|
| | ``get()`` |各要素にインデックスを付ける |
| |スライス() |各要素をスライス|
| | ``slice_replace()`` |各要素のスライスを渡された値に置き換えます|
| | ``cat()`` |文字列を連結|
| | ``repeat()`` |繰り返し値 |
| | ``normalize()`` |文字列の Unicode 形式を返します |
| | ``pad()`` |文字列の左、右、または両側に空白を追加|
| | ``wrap()`` |長い文字列を指定された幅よりも短い長さの行に分割します|
| | ``join()`` | Series の各要素の文字列を、渡されたセパレータで結合します|
| | get_dummies() |ダミー変数をデータフレームとして抽出 |

#### ベクトル化されたアイテムへのアクセスとスライス

特に ``get()`` および ``slice()`` 操作は、各配列からベクトル化された要素へのアクセスを有効にします。
たとえば、 str.slice(0, 3) を使用して、各配列の最初の 3 文字のスライスを取得できます。
この動作は、Python の通常のインデックス構文でも利用できることに注意してください。たとえば、 df.str.slice(0, 3) は df.str[0:3] と同等です。

In [13]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

``df.str.get(i)`` と ``df.str[i]`` によるインデックス作成も同様です。

これらの ``get()`` および ``slice()`` メソッドは、``split()`` によって返された配列の要素にアクセスすることもできます。
たとえば、各エントリの姓を抽出するには、split() と get() を組み合わせることができます。

In [14]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

#### 指標変数

少し余分な説明が必要な別のメソッドは ``get_dummies()`` メソッドです。
これは、データに何らかのコード化されたインジケーターを含む列がある場合に役立ちます。
たとえば、A="アメリカ生まれ"、B="イギリス生まれ"、C="チーズ好き"、D="スパム好き" などのコード形式の情報を含むデータセットがあるとします。 :

In [15]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,info,name
0,B|C|D,Graham Chapman
1,B|D,John Cleese
2,A|C,Terry Gilliam
3,B|D,Eric Idle
4,B|C,Terry Jones
5,B|C|D,Michael Palin


``get_dummies()`` ルーチンを使用すると、これらのインジケータ変数を ``DataFrame`` にすばやく分割できます。

In [16]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


これらの操作をビルディング ブロックとして使用すると、データをクリーニングするときに無限の範囲の文字列処理手順を構築できます。

ここではこれらのメソッドについて詳しく説明しませんが、Pandas オンライン ドキュメントの ["Working with Text Data"](http://pandas.pydata.org/pandas-docs/stable/text.html) を読むか、["Working with Text Data"](http://pandas.pydata.org/pandas-docs/stable/text.html) にリストされているリソースを参照することをお勧めします。

## 例: レシピデータベース

これらのベクトル化された文字列操作は、乱雑な実世界のデータをクリーンアップするプロセスで最も役立ちます。
ここでは、Web 上のさまざまなソースからコンパイルされたオープンなレシピ データベースを使用して、その例を説明します。
私たちの目標は、レシピ データを成分リストに解析して、手元にあるいくつかの成分に基づいてレシピをすばやく見つけることです。

これをコンパイルするために使用されるスクリプトは https://github.com/fictivekin/openrecipes で見つけることができ、データベースの現在のバージョンへのリンクもそこにあります。

2016 年春の時点で、このデータベースは約 30 MB であり、次のコマンドでダウンロードして解凍できます。

In [17]:
# !curl -O http://openrecipes.s3.amazonaws.com/recipeitems-latest.json.gz
# !gunzip recipeitems-latest.json.gz

データベースは JSON 形式なので、 pd.read_json で読み取ってみます:

In [18]:
try:
    recipes = pd.read_json('recipeitems-latest.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


おっとっと！ 「末尾のデータ」があることを示す ``ValueError`` が返されます。
インターネットでこのエラーのテキストを検索すると、*各行*自体が有効な JSON であるファイルを使用しているようですが、完全なファイルはそうではありません。
この解釈が正しいかどうかを確認しましょう。

In [19]:
with open('recipeitems-latest.json') as f:
    line = f.readline()
pd.read_json(line).shape

(2, 12)

はい、どうやら各行は有効な JSON であるため、それらをつなぎ合わせる必要があります。
これを行う方法の 1 つは、これらすべての JSON エントリを含む文字列表現を実際に作成し、すべてを ``pd.read_json`` でロードすることです。

In [20]:
# read the entire file into a Python array
with open('recipeitems-latest.json', 'r') as f:
    # Extract each line
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON
recipes = pd.read_json(data_json)

In [21]:
recipes.shape

(173278, 17)

約 200,000 のレシピと 17 の列があることがわかります。
1 つの行を見て、何があるかを見てみましょう。

In [22]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
cookTime                                                          PT30M
creator                                                             NaN
dateModified                                                        NaN
datePublished                                                2013-03-11
description           Late Saturday afternoon, after Marlboro Man ha...
image                 http://static.thepioneerwoman.com/cooking/file...
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
name                                    Drop Biscuits and Sausage Gravy
prepTime                                                          PT10M
recipeCategory                                                      NaN
recipeInstructions                                                  NaN
recipeYield                                                          12
source                                                  thepione

そこには多くの情報がありますが、Web からスクレイピングされたデータによく見られるように、その多くは非常に乱雑な形式になっています。
特に、成分リストは文字列形式です。関心のある情報を注意深く抽出する必要があります。
成分を詳しく見てみましょう。

In [23]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

成分リストの長さは平均 250 文字で、最小文字数は 0 文字、最大文字数は約 10,000 文字です。

興味本位で、材料リストが最も長いレシピを見てみましょう。

In [24]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

それは確かに複雑なレシピのように見えます.

他の集約探索を行うことができます。たとえば、朝食用のレシピがいくつあるか見てみましょう。

In [25]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

または、シナモンを材料として挙げているレシピはいくつありますか。

In [26]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

材料のスペルが「シナモン」と間違っているレシピがないかどうかを確認することもできます。

In [27]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

これは、Pandas 文字列ツールで可能なタイプの重要なデータ探索です。
Python が本当に得意とするのは、このようなデータマンギングです。

### シンプルなレシピレコメンデーション

もう少し進んで、簡単なレシピ推奨システムの作業を始めましょう。材料のリストが与えられた場合、それらすべての材料を使用するレシピを見つけます。
概念的には簡単ですが、データの不均一性によってタスクが複雑になります。たとえば、各行から成分のクリーンなリストを抽出するなどの簡単な操作はありません。
そこで、少しごまかすことにします。一般的な材料のリストから始めて、それらが各レシピの材料リストにあるかどうかを検索するだけです。
簡単にするために、当面はハーブとスパイスに固執しましょう。

In [28]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

次に、この成分がリストに表示されるかどうかを示す、True 値と False 値で構成されるブール値の ``DataFrame`` を構築できます。

In [29]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE))
                             for spice in spice_list))
spice_df.head()

,cumin,oregano,paprika,parsley,pepper,rosemary,sage,salt,tarragon,thyme
0,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


例として、パセリ、パプリカ、タラゴンを使ったレシピを見つけたいとしましょう。
[High-Performance Pandas: ``eval() と query() で説明されている ``DataFrame``s の ``query()`` メソッドを使用して、これを非常に迅速に計算できます](03.12-Performance-Eval-and-Query.ipynb ):

In [30]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

この組み合わせのレシピは 10 個しか見つかりません。この選択によって返されたインデックスを使用して、この組み合わせを持つレシピの名前を見つけてみましょう。

In [31]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

レシピの選択をほぼ 20,000 分の 1 に絞り込んだので、夕食に何を作りたいかについて、より多くの情報に基づいた決定を下すことができます。

### レシピをさらに進める

この例で、Pandas の文字列メソッドによって効率的に有効化されるデータ クリーニング操作のタイプについて少し理解できたことを願っています (ba-dum!)。
もちろん、非常に堅牢なレシピ推奨システムを構築するには、さらに多くの作業が必要になります!
各レシピから完全な成分リストを抽出することは、タスクの重要な部分です。残念ながら、使用されるフォーマットが多種多様であるため、このプロセスは比較的時間がかかります。
これは、データ サイエンスでは、現実世界のデータのクリーニングと変更が多くの​​場合、作業の大部分を占めるという自明の理を示しています。Pandas は、これを効率的に行うのに役立つツールを提供します。

<!--ナビゲーション-->
< [Pivot Tables](03.09-Pivot-Tables.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.10-Working-With-Strings.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
